# Overview
- bssidの位置特定

In [1]:
import subprocess
cmd = "git rev-parse --short HEAD"
hash = subprocess.check_output(cmd.split()).strip().decode('utf-8')
print(hash)

df5eb02


# Const

In [2]:
NB = '013'
DIR_TRAIN = './../data_ignore/input/train/'
DIR_TEST = './../data_ignore/input/test/'
DIR_WIFI = './../data_ignore/input/wifi/'
PATH_SUB = './../data_ignore/input/sample_submission.csv'
PATH_99_SUB = './../data/input/floor_99per_acc_sub.csv'
DIR_SAVE_IGNORE = f'./../data_ignore/nb/{NB}/'
DIR_SAVE = f'./../data/nb/{NB}/'

# Import everything I need:)

In [3]:
import os
import time
import yaml
import json
import types
import random
import pickle
import builtins
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from icecream import ic
from ipdb import set_trace as st
import matplotlib.pyplot as plt
from dataclasses import dataclass
# from tqdm import tqdm
from fastprogress import progress_bar, master_bar
from glob import glob
from loguru import logger
from collections import OrderedDict
from lmfit import Parameters, Minimizer, report_fit


# Function

In [4]:
def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )


def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)

    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

In [6]:
def trace_plot(site, floor):
    # trace
    path_trace_list = sorted(glob(f'./../data_ignore/input/train/{site}/{floor}/*'))

    # json
    path_json = f'./../data_ignore/input/metadata/{site}/{floor}/floor_info.json'
    with open(path_json) as json_file:
        json_data = json.load(json_file)
    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    # show image
    title = f'site: {site}\nfloor: {floor}'
    cmap = plt.get_cmap("tab20b")
    path_img = f'./../data_ignore/input/metadata/{site}/{floor}/floor_image.png'
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    fig.patch.set_facecolor('white')
    fig.suptitle(title, fontsize=18)
    im = Image.open(path_img)
    ax.imshow(np.asarray(im), extent=(0, width_meter, 0, height_meter))

    for i_trace in range(len(path_trace_list)):
        data = read_data_file(path_trace_list[i_trace])
        waypoints = data.waypoint
        x_arr = waypoints[:, 1]
        y_arr = waypoints[:, 2]

        ax.plot(x_arr, y_arr, linewidth=2, alpha=0.5)
    return fig

In [7]:
def get_site_floor_fig(site, floor):
    # json
    path_json = f'./../data_ignore/input/metadata/{site}/{floor}/floor_info.json'
    with open(path_json) as json_file:
        json_data = json.load(json_file)
    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]
    s = max(width_meter, height_meter)
    w = round(10*width_meter/s)
    h = round(10*height_meter/s)

    # show image
    cmap = plt.get_cmap("tab20b")
    path_img = f'./../data_ignore/input/metadata/{site}/{floor}/floor_image.png'
    fig, ax = plt.subplots(1, 1, figsize=(w, h))
    fig.patch.set_facecolor('white')
    title = f'site: {site}\nfloor: {floor}'
    fig.suptitle(title, fontsize=18)
    im = Image.open(path_img)
    ax.imshow(np.asarray(im), extent=(0, width_meter, 0, height_meter))
    return fig, ax, width_meter, height_meter


def gaussian2D(x, y, height, cen_x, cen_y, sig):
    return height*np.exp(-(((cen_x-x)/sig)**2 + ((cen_y-y)/sig)**2)/2.0)


def residuals(p, x, y, z):
    height = p["height"].value
    cen_x = p["centroid_x"].value
    cen_y = p["centroid_y"].value
    sigma = p["sig"].value
    return (z - gaussian2D(x,y,height,cen_x, cen_y, sigma))

In [8]:
@noglobal
def fitting(x_arr, y_arr, rssi_arr, n_grid, width_meter, height_meter):
    gauss = np.zeros([n_grid, n_grid])
    x, y = np.meshgrid(np.linspace(0, width_meter, n_grid), np.linspace(0, height_meter, n_grid))
    c = np.exp(np.max(rssi_arr))
    for i in range(len(x_arr)):
        gauss += gaussian2D(x, y, 1/c*np.exp(rssi_arr[i]), x_arr[i], y_arr[i], 20)

    initial = Parameters()
    initial.add("height", value=np.max(1/c*np.exp(rssi_arr)), min=0.5)
    initial.add("centroid_x", value=width_meter/2, min=0, max=width_meter)
    initial.add("centroid_y", value=height_meter/2, min=0, max=height_meter)
    initial.add("sig", value=100)

    fit = Minimizer(residuals, initial, fcn_args=(x, y, gauss))
    out = fit.leastsq()
    best_params = [param.value for _, param in out.params.items()]   # height, centroid_x, centroid_y, sig
    best_fit = gaussian2D(x, y, best_params[0], best_params[1], best_params[2], best_params[3])
    return best_params, best_fit


@noglobal
def get_near_waypoints(df_wifi_ssid_bssid, df_waypoint, sec):
    ts_wp = df_waypoint['timestamp'].values
    x_arr = []
    y_arr = []
    rssi_arr = []
    for idx, row in df_wifi_ssid_bssid.iterrows():
        lastseen_ts = row.lastseen_ts
        rssi = row['rssi']
        logics = (lastseen_ts/1000.0 - sec <= ts_wp/1000.0) & (ts_wp/1000.0 < lastseen_ts/1000.0 + sec)
        if np.sum(logics) != 0:
            _x_arr = df_waypoint[logics].x
            _y_arr = df_waypoint[logics].y
            _rssi_arr = [rssi]*len(_x_arr)
            x_arr = x_arr + _x_arr.tolist()
            y_arr = y_arr + _y_arr.tolist()
            rssi_arr = rssi_arr + _rssi_arr

    df = pd.DataFrame({'x': x_arr, 'y': y_arr, 'rssi': rssi_arr})
    df = df.groupby(['x', 'y']).mean().reset_index()
    x_arr = df.x.values
    y_arr = df.y.values
    rssi_arr = df.rssi.values
    return x_arr, y_arr, rssi_arr

# Preparation

<br>

set

In [9]:
pd.set_option('display.max_rows', 500)

if not os.path.exists(DIR_SAVE_IGNORE):
    os.makedirs(DIR_SAVE_IGNORE)
if not os.path.exists(DIR_SAVE):
    os.makedirs(DIR_SAVE)

<br>

load dataset

In [10]:
sample_submission = pd.read_csv(PATH_SUB)

# Save wifi waypoint

In [11]:
n_grid = 100
sec = 5

In [61]:
%%time

site_list = [val.split('_')[0] for val in sample_submission.site_path_timestamp]
site_list = sorted(np.unique(site_list).tolist())

# site loop
# for site in progress_bar(site_list):
for site in site_list:
# for site in site_list[:2]:
    # init
    site_list_save = []
    floor_list_save = []
    ssid_list_save = []
    bssid_list_save = []
    n_samples_list_save = []
    n_samples_rssi_over_m50_list_save = []
    n_samples_rssi_over_m55_list_save = []
    n_samples_rssi_over_m60_list_save = []
    n_samples_rssi_over_m65_list_save = []
    n_samples_rssi_over_m70_list_save = []
    bssid_x_list_save = []
    bssid_y_list_save = []
    
    # dataのロード
    path_waypoint = f'./../data_ignore/nb/010/nb010_waypoint_{site}.csv'
    df_wp_ori = pd.read_csv(path_waypoint)

    path_wifi = f'./../data_ignore/nb/004/nb004_wifi_{site}.csv'
    df_wifi_ori = pd.read_csv(path_wifi)
    
    # floorのリストの取得
    floor_list = sorted(glob(f'./../data_ignore/input/train/{site}/*'))
    floor_list = [text.split('/')[-1] for text in floor_list]
    
    # floor loop
    for floor in floor_list:
#     for floor in floor_list[:1]:
        print(f'{site}-[{floor}] start at {time.ctime()}')
        path_json = f'./../data_ignore/input/metadata/{site}/{floor}/floor_info.json'
        with open(path_json) as json_file:
            json_data = json.load(json_file)
        width_meter = json_data["map_info"]["width"]
        height_meter = json_data["map_info"]["height"]
        
        df_wp = df_wp_ori.query('floor == @floor')
        df_wifi = df_wifi_ori.query('floor == @floor')
        
        # ssid loop
        mb = master_bar(df_wifi['ssid '].unique())
        for ssid in mb:
            df_wifi_ssid = df_wifi[df_wifi['ssid '] == ssid]
            
            # bssid loop
            for bssid in progress_bar(df_wifi_ssid['bssid'].unique(), parent=mb):
#             for bssid in progress_bar(df_wifi_ssid['bssid'].unique()[:2], parent=mb):
                df_wifi_ssid_bssid = df_wifi_ssid.query('bssid == @bssid')
                # site, floor, ssid, bssidが決まった！
                # あるwifiのlast_seen_timestamp の前後 sec 秒以内 のwaypointを取得する
                x_arr, y_arr, rssi_arr = get_near_waypoints(df_wifi_ssid_bssid, df_wp, sec)
                if len(x_arr)==0:
                    continue

                # ガウス分布でフィッティング
                best_params, best_fit = fitting(x_arr, y_arr, rssi_arr, n_grid, width_meter, height_meter)
                
                # append
                site_list_save.append(site)
                floor_list_save.append(floor)
                ssid_list_save.append(ssid)
                bssid_list_save.append(bssid)
                n_samples_list_save.append(len(rssi_arr))
                n_samples_rssi_over_m50_list_save.append((rssi_arr>=-50).sum())
                n_samples_rssi_over_m55_list_save.append((rssi_arr>=-55).sum())
                n_samples_rssi_over_m60_list_save.append((rssi_arr>=-60).sum())
                n_samples_rssi_over_m65_list_save.append((rssi_arr>=-65).sum())
                n_samples_rssi_over_m70_list_save.append((rssi_arr>=-70).sum())
                bssid_x_list_save.append(best_params[1])
                bssid_y_list_save.append(best_params[2])
                
    # site毎に保存
    df_save = pd.DataFrame({
        'ssid': ssid_list_save,
        'bssid': bssid_list_save,
        'bssid_x': bssid_x_list_save,
        'bssid_y': bssid_y_list_save,
        'n_samples': n_samples_list_save,
        'n_samples_rssi_over_m50': n_samples_rssi_over_m50_list_save,
        'n_samples_rssi_over_m55': n_samples_rssi_over_m55_list_save,
        'n_samples_rssi_over_m60': n_samples_rssi_over_m60_list_save,
        'n_samples_rssi_over_m65': n_samples_rssi_over_m65_list_save,
        'n_samples_rssi_over_m70': n_samples_rssi_over_m70_list_save,
        'site': site_list_save,
        'floor': floor_list_save,
    })
    save_path = f'{DIR_SAVE}nb{NB}_bssid_position_{site}.csv'
    df_save.to_csv(save_path, index=False)

5a0546857ecc773753327266-[B1] start at Fri Apr  2 15:04:09 2021


5a0546857ecc773753327266-[F1] start at Fri Apr  2 15:05:03 2021


5a0546857ecc773753327266-[F2] start at Fri Apr  2 15:06:51 2021


5a0546857ecc773753327266-[F3] start at Fri Apr  2 15:09:08 2021


5a0546857ecc773753327266-[F4] start at Fri Apr  2 15:10:11 2021


...

5dc8cea7659e181adb076a3f-[F4] start at Fri Apr  2 17:16:55 2021


5dc8cea7659e181adb076a3f-[F5] start at Fri Apr  2 17:17:51 2021


5dc8cea7659e181adb076a3f-[F6] start at Fri Apr  2 17:19:00 2021


5dc8cea7659e181adb076a3f-[F7] start at Fri Apr  2 17:19:42 2021


CPU times: user 2h 16min 1s, sys: 33.1 s, total: 2h 16min 34s
Wall time: 2h 15min 52s
